In [1]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
from datetime import datetime

def connect_to_database(db_path='../database/option_strategies.db'):
    """Connect to the SQLite database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Verify the database has the required table
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='option_strategies'")
        if not cursor.fetchone():
            print(f"Error: Database {db_path} does not contain the option_strategies table.")
            print("Please run the database setup script first.")
            conn.close()
            return None, None
            
        return conn, cursor
    except Exception as e:
        print(f"Error connecting to database: {str(e)}")
        return None, None

def extract_date(driver):
    """Extract the date from the page"""
    try:
        # Search the page text
        page_text = driver.find_element(By.TAG_NAME, "body").text
        date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d+\w*,\s+\d{4}'
        matches = re.findall(date_pattern, page_text)
        
        if matches:
            return matches[0]
        
        return "Date not found"
        
    except Exception as e:
        print(f"Error extracting date: {str(e)}")
        return "Date extraction error"

def extract_options_expiry_date(driver, tab_content=None):
    """Extract the Options Expiry Date"""
    try:
        # Search entire page
        page_text = driver.find_element(By.TAG_NAME, "body").text
        date_match = re.search(r'Options Expiry Date:?\s*(\d{4}-\d{2}-\d{2})', page_text)
        if date_match:
            return date_match.group(1)
        
        # More general search
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', page_text)
        if date_match:
            return date_match.group(1)
        
        return "Expiry date not found"
        
    except Exception as e:
        print(f"Error extracting options expiry date: {str(e)}")
        return "Expiry date extraction error"

def extract_table_data(driver, tab, tab_index, date_info, strategy_type, conn, cursor):
    """Extract data from the table in the current tab and save to database"""
    try:
        tab_name = tab.text.strip().replace('\n', ' ')
        print(f"Processing Tab {tab_index+1}: '{tab_name}'")
        
        # Click the tab
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(1)
        
        try:
            tab.click()
        except:
            driver.execute_script("arguments[0].click();", tab)
        
        time.sleep(2)
        
        # Find tab content and table
        tab_href = tab.get_attribute("href")
        tab_content = None
        table = None
        
        if tab_href and "#" in tab_href:
            tab_id = tab_href.split("#")[1]
            try:
                tab_content = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, tab_id))
                )
                
                # Extract options expiry date
                options_expiry_date = extract_options_expiry_date(driver, tab_content)
                
                # Find table inside tab content
                tables_in_tab = tab_content.find_elements(By.TAG_NAME, "table")
                if tables_in_tab:
                    table = tables_in_tab[0]
            except Exception:
                pass
        
        # If no table found, look for any table
        if not table:
            tables = driver.find_elements(By.TAG_NAME, "table")
            visible_tables = [t for t in tables if t.is_displayed()]
            table = visible_tables[0] if visible_tables else tables[0] if tables else None
        
        if not table:
            print(f"No tables found in tab #{tab_index+1}")
            return 0
        
        # Extract headers
        headers = table.find_elements(By.TAG_NAME, "th")
        header_texts = [header.text.strip() for header in headers]
        
        # Find column indices
        column_map = {
            'ID': next((i for i, h in enumerate(header_texts) if 'ID' in h.upper()), -1),
            'Ticker': next((i for i, h in enumerate(header_texts) if 'TICKER' in h.upper() or 'SYMBOL' in h.upper()), -1),
            'Trigger Price': next((i for i, h in enumerate(header_texts) if ('TRIGGER' in h.upper() and 'PRICE' in h.upper()) or 'ENTRY' in h.upper()), -1),
            'Strike Price': next((i for i, h in enumerate(header_texts) if 'STRIKE' in h.upper() and 'PRICE' in h.upper()), -1),
            'Estimated Premium': next((i for i, h in enumerate(header_texts) if 'PREMIUM' in h.upper() or 'ESTIMATED' in h.upper()), -1)
        }
        
        # Extract rows
        rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header
        records_count = 0
        
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if not cells:
                continue
            
            # Extract data from cells
            # Extract data from cells
            item_id = cells[column_map['ID']].text.strip() if column_map['ID'] != -1 and column_map['ID'] < len(cells) else 'N/A'
            ticker_raw = cells[column_map['Ticker']].text.strip() if column_map['Ticker'] != -1 and column_map['Ticker'] < len(cells) else 'N/A'

            # Check if ticker contains (ER) and process accordingly
            er_value = 0
            if "(ER)" in ticker_raw:
                ticker = ticker_raw.replace("(ER)", "").strip()
                er_value = 1
            else:
                ticker = ticker_raw

            trigger_price = cells[column_map['Trigger Price']].text.strip() if column_map['Trigger Price'] != -1 and column_map['Trigger Price'] < len(cells) else 'N/A'
            strike_price = cells[column_map['Strike Price']].text.strip() if column_map['Strike Price'] != -1 and column_map['Strike Price'] < len(cells) else 'N/A'
            estimated_premium = cells[column_map['Estimated Premium']].text.strip() if column_map['Estimated Premium'] != -1 and column_map['Estimated Premium'] < len(cells) else 'N/A'

            # Get options expiry date if it exists
            expiry_date = options_expiry_date if 'options_expiry_date' in locals() else 'N/A'

            # Parse 'strike_price' to extract 'buy' and 'sell' values
            strike_buy_value, strike_sell_value = 0.0, 0.0  # Default values in case of parsing failure
            if " - " in strike_price:
                parts = strike_price.split(" - ")
                if len(parts) == 2:
                    strike_sell_part = parts[0].strip()
                    strike_buy_part = parts[1].strip()

                    # Extract numerical values
                    strike_sell_value = float(strike_sell_part.split(" ")[1]) if "sell" in strike_sell_part.lower() else 0.0
                    strike_buy_value = float(strike_buy_part.split(" ")[1]) if "buy" in strike_buy_part.lower() else 0.0

            # Updated database insert command to include 'er' column
            cursor.execute('''
            INSERT INTO option_strategies (
                scrape_date, strategy_type, tab_name, ticker, trigger_price, 
                strike_price, strike_buy, strike_sell, estimated_premium, item_id, options_expiry_date, date_info, er
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                datetime.now().isoformat(), strategy_type, tab_name, ticker, trigger_price, 
                strike_price, strike_buy_value, strike_sell_value, estimated_premium, item_id, expiry_date, date_info, er_value
            ))

            conn.commit()
            records_count += 1
        
        print(f"Saved {records_count} records from tab #{tab_index+1}")
        return records_count
        
    except Exception as e:
        print(f"Error processing tab #{tab_index+1}: {str(e)}")
        return 0

def process_strategy_page(driver, strategy_url, strategy_type, conn, cursor):
    """Process a single strategy page and extract data from all tabs"""
    try:
        print(f"\n===== Processing {strategy_type} Strategy Page =====")
        driver.get(strategy_url)
        time.sleep(3)
        
        # Extract the date
        date_info = extract_date(driver)
        
        # Find all tabs using multiple methods
        tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'ep_tabs_header')]//a[contains(@class, 'ep_label_main')]")
        if not tabs:
            tabs = driver.find_elements(By.XPATH, "//a[contains(@class, 'ep_label_main')]")
        if not tabs:
            tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'tabs')]//a")
        
        if not tabs:
            print(f"No tab elements found on the {strategy_type} page")
            return 0
        
        # Process only the first 4 tabs
        num_tabs_to_process = min(4, len(tabs))
        total_records = 0
        
        for i, tab in enumerate(tabs[:num_tabs_to_process]):
            records = extract_table_data(driver, tab, i, date_info, strategy_type, conn, cursor)
            total_records += records
        
        return total_records
            
    except Exception as e:
        print(f"Error processing {strategy_type} strategy page: {str(e)}")
        return 0

def scrape_option_strategies(db_path='../database/option_strategies.db', browser_type="chrome", manual_login_wait_time=30):
    """
    Scrape data from option strategy pages and save to SQLite database
    
    Parameters:
    db_path (str): Path to the SQLite database file
    browser_type (str): 'chrome' or 'edge'
    manual_login_wait_time (int): Number of seconds to wait for manual login
    
    Returns:
    int: Number of records added to the database
    """
    
    # Connect to the database
    conn, cursor = connect_to_database(db_path)
    if not conn or not cursor:
        return 0
    
    # Initialize browser
    chrome_options = ChromeOptions()
    chrome_options.add_argument('--start-maximized')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        # Start login process
        print("Starting login process...")
        driver.get("https://optionrecom.com/my-account-2/")
        
        print(f"\nPlease log in manually. Waiting {manual_login_wait_time} seconds...")
        time.sleep(manual_login_wait_time)
        
        # Define strategies to scrape
        strategies = [
            {
                "url": "https://optionrecom.com/bear-call-spread-strategy/",
                "type": "Bear Call"
            },
            {
                "url": "https://optionrecom.com/bull-put-spread-strategy/",
                "type": "Bull Put"
            }
        ]
        
        # Process each strategy page
        total_records = 0
        
        for strategy in strategies:
            records = process_strategy_page(driver, strategy["url"], strategy["type"], conn, cursor)
            total_records += records
        
        print(f"\nTotal records saved to database: {total_records}")
        
        # Query to show what was saved
        cursor.execute("SELECT strategy_type, tab_name, COUNT(*) as count FROM option_strategies GROUP BY strategy_type, tab_name")
        results = cursor.fetchall()
        
        print("\nRecords by strategy and tab:")
        for strategy, tab, count in results:
            print(f"  {strategy} - {tab}: {count} records")
        
        return total_records
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return 0
    finally:
        driver.quit()
        conn.close()

if __name__ == "__main__":
    scrape_option_strategies()

Starting login process...

Please log in manually. Waiting 30 seconds...

===== Processing Bear Call Strategy Page =====
Processing Tab 1: 'Mild Risk 95-97% accuracy > shorter expiry'
Saved 3 records from tab #1
Processing Tab 2: 'Minimal Risk 97-99% accuracy > shorter expiry'
Saved 1 records from tab #2
Processing Tab 3: 'Mild Risk 95-97% accuracy > longer expiry'
Saved 1 records from tab #3
Processing Tab 4: 'Minimal Risk 97-99% accuracy > longer expiry'
Saved 1 records from tab #4

===== Processing Bull Put Strategy Page =====
Processing Tab 1: 'Mild Risk 95-97% accuracy > shorter expiry'
Saved 2 records from tab #1
Processing Tab 2: 'Minimal Risk 97-99% accuracy > shorter expiry'
Saved 1 records from tab #2
Processing Tab 3: 'Mild Risk 95-97% accuracy > longer expiry'
Saved 1 records from tab #3
Processing Tab 4: 'Minimal Risk 97-99% accuracy > longer expiry'
Saved 2 records from tab #4

Total records saved to database: 12

Records by strategy and tab:
  Bear Call - Mild Risk 95-97

## this one works

In [2]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
from datetime import datetime

def connect_to_database(db_path='../database/option_strategies.db'):
    """Connect to the SQLite database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Verify the database has the required table
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='option_strategies'")
        if not cursor.fetchone():
            print(f"Error: Database {db_path} does not contain the option_strategies table.")
            print("Please run the database setup script first.")
            conn.close()
            return None, None
            
        return conn, cursor
    except Exception as e:
        print(f"Error connecting to database: {str(e)}")
        return None, None

def extract_date(driver):
    """Extract the date from the page"""
    try:
        # Search the page text
        page_text = driver.find_element(By.TAG_NAME, "body").text
        date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d+\w*,\s+\d{4}'
        matches = re.findall(date_pattern, page_text)
        
        if matches:
            return matches[0]
        
        return "Date not found"
        
    except Exception as e:
        print(f"Error extracting date: {str(e)}")
        return "Date extraction error"

def extract_options_expiry_date(driver, tab_content=None):
    """Extract the Options Expiry Date"""
    try:
        # Search entire page
        page_text = driver.find_element(By.TAG_NAME, "body").text
        date_match = re.search(r'Options Expiry Date:?\s*(\d{4}-\d{2}-\d{2})', page_text)
        if date_match:
            return date_match.group(1)
        
        # More general search
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', page_text)
        if date_match:
            return date_match.group(1)
        
        return "Expiry date not found"
        
    except Exception as e:
        print(f"Error extracting options expiry date: {str(e)}")
        return "Expiry date extraction error"

def extract_table_data(driver, tab, tab_index, date_info, strategy_type, conn, cursor):
    """Extract data from the table in the current tab and save to database"""
    try:
        tab_name = tab.text.strip().replace('\n', ' ')
        print(f"Processing Tab {tab_index+1}: '{tab_name}'")
        
        # Click the tab
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(1)
        
        try:
            tab.click()
        except:
            driver.execute_script("arguments[0].click();", tab)
        
        time.sleep(2)
        
        # Find tab content and table
        tab_href = tab.get_attribute("href")
        tab_content = None
        table = None
        
        if tab_href and "#" in tab_href:
            tab_id = tab_href.split("#")[1]
            try:
                tab_content = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, tab_id))
                )
                
                # Extract options expiry date
                options_expiry_date = extract_options_expiry_date(driver, tab_content)
                
                # Find table inside tab content
                tables_in_tab = tab_content.find_elements(By.TAG_NAME, "table")
                if tables_in_tab:
                    table = tables_in_tab[0]
            except Exception:
                pass
        
        # If no table found, look for any table
        if not table:
            tables = driver.find_elements(By.TAG_NAME, "table")
            visible_tables = [t for t in tables if t.is_displayed()]
            table = visible_tables[0] if visible_tables else tables[0] if tables else None
        
        if not table:
            print(f"No tables found in tab #{tab_index+1}")
            return 0
        
        # Extract headers
        headers = table.find_elements(By.TAG_NAME, "th")
        header_texts = [header.text.strip() for header in headers]
        
        # Find column indices
        column_map = {
            'ID': next((i for i, h in enumerate(header_texts) if 'ID' in h.upper()), -1),
            'Ticker': next((i for i, h in enumerate(header_texts) if 'TICKER' in h.upper() or 'SYMBOL' in h.upper()), -1),
            'Trigger Price': next((i for i, h in enumerate(header_texts) if ('TRIGGER' in h.upper() and 'PRICE' in h.upper()) or 'ENTRY' in h.upper()), -1),
            'Strike Price': next((i for i, h in enumerate(header_texts) if 'STRIKE' in h.upper() and 'PRICE' in h.upper()), -1),
            'Estimated Premium': next((i for i, h in enumerate(header_texts) if 'PREMIUM' in h.upper() or 'ESTIMATED' in h.upper()), -1)
        }
        
        # Extract rows
        rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header
        records_count = 0
        
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if not cells:
                continue
            
            # Extract data from cells
            item_id = cells[column_map['ID']].text.strip() if column_map['ID'] != -1 and column_map['ID'] < len(cells) else 'N/A'
            ticker = cells[column_map['Ticker']].text.strip() if column_map['Ticker'] != -1 and column_map['Ticker'] < len(cells) else 'N/A'
            trigger_price = cells[column_map['Trigger Price']].text.strip() if column_map['Trigger Price'] != -1 and column_map['Trigger Price'] < len(cells) else 'N/A'
            strike_price = cells[column_map['Strike Price']].text.strip() if column_map['Strike Price'] != -1 and column_map['Strike Price'] < len(cells) else 'N/A'
            estimated_premium = cells[column_map['Estimated Premium']].text.strip() if column_map['Estimated Premium'] != -1 and column_map['Estimated Premium'] < len(cells) else 'N/A'
            
            # Get options expiry date if it exists
            expiry_date = options_expiry_date if 'options_expiry_date' in locals() else 'N/A'
            
            # Parse 'strike_price' to extract 'buy' and 'sell' values
            strike_buy_value, strike_sell_value = 0.0, 0.0  # Default values in case of parsing failure
            if " - " in strike_price:
                parts = strike_price.split(" - ")
                if len(parts) == 2:
                    strike_sell_part = parts[0].strip()
                    strike_buy_part = parts[1].strip()

                    # Extract numerical values
                    strike_sell_value = float(strike_sell_part.split(" ")[1]) if "sell" in strike_sell_part.lower() else 0.0
                    strike_buy_value = float(strike_buy_part.split(" ")[1]) if "buy" in strike_buy_part.lower() else 0.0

            # Updated database insert command to include 'strike_sell' and 'strike_buy'
            cursor.execute('''
            INSERT INTO option_strategies (
                scrape_date, strategy_type, tab_name, ticker, trigger_price, 
                strike_price, strike_buy, strike_sell, estimated_premium, item_id, options_expiry_date, date_info
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                datetime.now().isoformat(), strategy_type, tab_name, ticker, trigger_price, 
                strike_price, strike_buy_value, strike_sell_value, estimated_premium, item_id, expiry_date, date_info
            ))
            
            conn.commit()
            records_count += 1
        
        print(f"Saved {records_count} records from tab #{tab_index+1}")
        return records_count
        
    except Exception as e:
        print(f"Error processing tab #{tab_index+1}: {str(e)}")
        return 0

def process_strategy_page(driver, strategy_url, strategy_type, conn, cursor):
    """Process a single strategy page and extract data from all tabs"""
    try:
        print(f"\n===== Processing {strategy_type} Strategy Page =====")
        driver.get(strategy_url)
        time.sleep(3)
        
        # Extract the date
        date_info = extract_date(driver)
        
        # Find all tabs using multiple methods
        tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'ep_tabs_header')]//a[contains(@class, 'ep_label_main')]")
        if not tabs:
            tabs = driver.find_elements(By.XPATH, "//a[contains(@class, 'ep_label_main')]")
        if not tabs:
            tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'tabs')]//a")
        
        if not tabs:
            print(f"No tab elements found on the {strategy_type} page")
            return 0
        
        # Process only the first 4 tabs
        num_tabs_to_process = min(4, len(tabs))
        total_records = 0
        
        for i, tab in enumerate(tabs[:num_tabs_to_process]):
            records = extract_table_data(driver, tab, i, date_info, strategy_type, conn, cursor)
            total_records += records
        
        return total_records
            
    except Exception as e:
        print(f"Error processing {strategy_type} strategy page: {str(e)}")
        return 0

def scrape_option_strategies(db_path='../database/option_strategies.db', browser_type="chrome", manual_login_wait_time=30):
    """
    Scrape data from option strategy pages and save to SQLite database
    
    Parameters:
    db_path (str): Path to the SQLite database file
    browser_type (str): 'chrome' or 'edge'
    manual_login_wait_time (int): Number of seconds to wait for manual login
    
    Returns:
    int: Number of records added to the database
    """
    
    # Connect to the database
    conn, cursor = connect_to_database(db_path)
    if not conn or not cursor:
        return 0
    
    # Initialize browser
    chrome_options = ChromeOptions()
    chrome_options.add_argument('--start-maximized')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        # Start login process
        print("Starting login process...")
        driver.get("https://optionrecom.com/my-account-2/")
        
        print(f"\nPlease log in manually. Waiting {manual_login_wait_time} seconds...")
        time.sleep(manual_login_wait_time)
        
        # Define strategies to scrape
        strategies = [
            {
                "url": "https://optionrecom.com/bear-call-spread-strategy/",
                "type": "Bear Call"
            },
            {
                "url": "https://optionrecom.com/bull-put-spread-strategy/",
                "type": "Bull Put"
            }
        ]
        
        # Process each strategy page
        total_records = 0
        
        for strategy in strategies:
            records = process_strategy_page(driver, strategy["url"], strategy["type"], conn, cursor)
            total_records += records
        
        print(f"\nTotal records saved to database: {total_records}")
        
        # Query to show what was saved
        cursor.execute("SELECT strategy_type, tab_name, COUNT(*) as count FROM option_strategies GROUP BY strategy_type, tab_name")
        results = cursor.fetchall()
        
        print("\nRecords by strategy and tab:")
        for strategy, tab, count in results:
            print(f"  {strategy} - {tab}: {count} records")
        
        return total_records
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return 0
    finally:
        driver.quit()
        conn.close()

if __name__ == "__main__":
    scrape_option_strategies()

Starting login process...

Please log in manually. Waiting 30 seconds...

===== Processing Bear Call Strategy Page =====
Processing Tab 1: 'Mild Risk 95-97% accuracy > shorter expiry'
Saved 2 records from tab #1
Processing Tab 2: 'Minimal Risk 97-99% accuracy > shorter expiry'
Saved 1 records from tab #2
Processing Tab 3: 'Mild Risk 95-97% accuracy > longer expiry'
Saved 1 records from tab #3
Processing Tab 4: 'Minimal Risk 97-99% accuracy > longer expiry'
Saved 1 records from tab #4

===== Processing Bull Put Strategy Page =====
Processing Tab 1: 'Mild Risk 95-97% accuracy > shorter expiry'
Saved 1 records from tab #1
Processing Tab 2: 'Minimal Risk 97-99% accuracy > shorter expiry'
Saved 1 records from tab #2
Processing Tab 3: 'Mild Risk 95-97% accuracy > longer expiry'
Saved 2 records from tab #3
Processing Tab 4: 'Minimal Risk 97-99% accuracy > longer expiry'
Saved 1 records from tab #4

Total records saved to database: 10

Records by strategy and tab:
  Bear Call - Mild Risk 95-97

In [2]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
from datetime import datetime

def connect_to_database(db_path='../database/option_strategies.db'):
    """Connect to the SQLite database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Verify the database has the required table
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='option_strategies'")
        if not cursor.fetchone():
            print(f"Error: Database {db_path} does not contain the option_strategies table.")
            print("Please run the database setup script first.")
            conn.close()
            return None, None
            
        return conn, cursor
    except Exception as e:
        print(f"Error connecting to database: {str(e)}")
        return None, None

def extract_date(driver):
    """Extract the date from the page"""
    try:
        # Search the page text
        page_text = driver.find_element(By.TAG_NAME, "body").text
        date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d+\w*,\s+\d{4}'
        matches = re.findall(date_pattern, page_text)
        
        if matches:
            return matches[0]
        
        return "Date not found"
        
    except Exception as e:
        print(f"Error extracting date: {str(e)}")
        return "Date extraction error"

def extract_options_expiry_date(driver, tab_content=None):
    """Extract the Options Expiry Date"""
    try:
        # Search entire page
        page_text = driver.find_element(By.TAG_NAME, "body").text
        date_match = re.search(r'Options Expiry Date:?\s*(\d{4}-\d{2}-\d{2})', page_text)
        if date_match:
            return date_match.group(1)
        
        # More general search
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', page_text)
        if date_match:
            return date_match.group(1)
        
        return "Expiry date not found"
        
    except Exception as e:
        print(f"Error extracting options expiry date: {str(e)}")
        return "Expiry date extraction error"

def extract_table_data(driver, tab, tab_index, date_info, strategy_type, conn, cursor):
    """Extract data from the table in the current tab and save to database"""
    try:
        tab_name = tab.text.strip().replace('\n', ' ')
        print(f"Processing Tab {tab_index+1}: '{tab_name}'")
        
        # Click the tab
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(1)
        
        try:
            tab.click()
        except:
            driver.execute_script("arguments[0].click();", tab)
        
        time.sleep(2)
        
        # Find tab content and table
        tab_href = tab.get_attribute("href")
        tab_content = None
        table = None
        
        if tab_href and "#" in tab_href:
            tab_id = tab_href.split("#")[1]
            try:
                tab_content = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, tab_id))
                )
                
                # Extract options expiry date
                options_expiry_date = extract_options_expiry_date(driver, tab_content)
                
                # Find table inside tab content
                tables_in_tab = tab_content.find_elements(By.TAG_NAME, "table")
                if tables_in_tab:
                    table = tables_in_tab[0]
            except Exception:
                pass
        
        # If no table found, look for any table
        if not table:
            tables = driver.find_elements(By.TAG_NAME, "table")
            visible_tables = [t for t in tables if t.is_displayed()]
            table = visible_tables[0] if visible_tables else tables[0] if tables else None
        
        if not table:
            print(f"No tables found in tab #{tab_index+1}")
            return 0, 0
        
        # Extract headers
        headers = table.find_elements(By.TAG_NAME, "th")
        header_texts = [header.text.strip() for header in headers]
        
        # Find column indices
        column_map = {
            'ID': next((i for i, h in enumerate(header_texts) if 'ID' in h.upper()), -1),
            'Ticker': next((i for i, h in enumerate(header_texts) if 'TICKER' in h.upper() or 'SYMBOL' in h.upper()), -1),
            'Trigger Price': next((i for i, h in enumerate(header_texts) if ('TRIGGER' in h.upper() and 'PRICE' in h.upper()) or 'ENTRY' in h.upper()), -1),
            'Strike Price': next((i for i, h in enumerate(header_texts) if 'STRIKE' in h.upper() and 'PRICE' in h.upper()), -1),
            'Estimated Premium': next((i for i, h in enumerate(header_texts) if 'PREMIUM' in h.upper() or 'ESTIMATED' in h.upper()), -1)
        }
        
        # Extract rows
        rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header
        records_count = 0
        skipped_records_count = 0  # Track skipped records
        
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if not cells:
                continue
            
            # Extract data from cells
            item_id = cells[column_map['ID']].text.strip() if column_map['ID'] != -1 and column_map['ID'] < len(cells) else 'N/A'
            
            # More careful handling of ticker value
            ticker_text = None
            if column_map['Ticker'] != -1 and column_map['Ticker'] < len(cells):
                ticker_text = cells[column_map['Ticker']].text.strip()
                # Check for none-like values (None, N/A, empty string)
                if not ticker_text or ticker_text.lower() == 'n/a' or ticker_text.lower() == 'none':
                    ticker_text = None
            
            trigger_price = cells[column_map['Trigger Price']].text.strip() if column_map['Trigger Price'] != -1 and column_map['Trigger Price'] < len(cells) else 'N/A'
            strike_price = cells[column_map['Strike Price']].text.strip() if column_map['Strike Price'] != -1 and column_map['Strike Price'] < len(cells) else 'N/A'
            estimated_premium = cells[column_map['Estimated Premium']].text.strip() if column_map['Estimated Premium'] != -1 and column_map['Estimated Premium'] < len(cells) else 'N/A'
            
            # Get options expiry date if it exists
            expiry_date = options_expiry_date if 'options_expiry_date' in locals() else 'N/A'
            
            # Skip records where ticker is None or empty
            if ticker_text is None:
                skipped_records_count += 1
                continue
            
            # Parse 'strike_price' to extract 'buy' and 'sell' values
            strike_buy_value, strike_sell_value = 0.0, 0.0  # Default values in case of parsing failure
            if " - " in strike_price:
                parts = strike_price.split(" - ")
                if len(parts) == 2:
                    strike_sell_part = parts[0].strip()
                    strike_buy_part = parts[1].strip()

                    # Extract numerical values
                    strike_sell_value = float(strike_sell_part.split(" ")[1]) if "sell" in strike_sell_part.lower() else 0.0
                    strike_buy_value = float(strike_buy_part.split(" ")[1]) if "buy" in strike_buy_part.lower() else 0.0

            # Double-check ticker before inserting
            if ticker_text is not None and ticker_text.strip() != '':
                # Updated database insert command to include 'strike_sell' and 'strike_buy'
                cursor.execute('''
                INSERT INTO option_strategies (
                    scrape_date, strategy_type, tab_name, ticker, trigger_price, 
                    strike_price, strike_buy, strike_sell, estimated_premium, item_id, options_expiry_date, date_info
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    datetime.now().isoformat(), strategy_type, tab_name, ticker_text, trigger_price, 
                    strike_price, strike_buy_value, strike_sell_value, estimated_premium, item_id, expiry_date, date_info
                ))
                
                conn.commit()
                records_count += 1
            else:
                # This is a failsafe in case the earlier check missed something
                skipped_records_count += 1
        
        print(f"Saved {records_count} records from tab #{tab_index+1}")
        if skipped_records_count > 0:
            print(f"Skipped {skipped_records_count} records with None ticker values from tab #{tab_index+1}")
            
        return records_count, skipped_records_count
        
    except Exception as e:
        print(f"Error processing tab #{tab_index+1}: {str(e)}")
        return 0, 0

def process_strategy_page(driver, strategy_url, strategy_type, conn, cursor):
    """Process a single strategy page and extract data from all tabs"""
    try:
        print(f"\n===== Processing {strategy_type} Strategy Page =====")
        driver.get(strategy_url)
        time.sleep(3)
        
        # Extract the date
        date_info = extract_date(driver)
        
        # Find all tabs using multiple methods
        tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'ep_tabs_header')]//a[contains(@class, 'ep_label_main')]")
        if not tabs:
            tabs = driver.find_elements(By.XPATH, "//a[contains(@class, 'ep_label_main')]")
        if not tabs:
            tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'tabs')]//a")
        
        if not tabs:
            print(f"No tab elements found on the {strategy_type} page")
            return 0, 0
        
        # Process only the first 4 tabs
        num_tabs_to_process = min(4, len(tabs))
        total_records = 0
        total_skipped = 0
        
        for i, tab in enumerate(tabs[:num_tabs_to_process]):
            records, skipped = extract_table_data(driver, tab, i, date_info, strategy_type, conn, cursor)
            total_records += records
            total_skipped += skipped
        
        return total_records, total_skipped
            
    except Exception as e:
        print(f"Error processing {strategy_type} strategy page: {str(e)}")
        return 0, 0

def scrape_option_strategies(db_path='../database/option_strategies.db', browser_type="chrome", manual_login_wait_time=30):
    """
    Scrape data from option strategy pages and save to SQLite database
    
    Parameters:
    db_path (str): Path to the SQLite database file
    browser_type (str): 'chrome' or 'edge'
    manual_login_wait_time (int): Number of seconds to wait for manual login
    
    Returns:
    int: Number of records added to the database
    """
    
    # Connect to the database
    conn, cursor = connect_to_database(db_path)
    if not conn or not cursor:
        return 0
    
    # Initialize browser
    chrome_options = ChromeOptions()
    chrome_options.add_argument('--start-maximized')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        # Start login process
        print("Starting login process...")
        driver.get("https://optionrecom.com/my-account-2/")
        
        print(f"\nPlease log in manually. Waiting {manual_login_wait_time} seconds...")
        time.sleep(manual_login_wait_time)
        
        # Define strategies to scrape
        strategies = [
            {
                "url": "https://optionrecom.com/bear-call-spread-strategy/",
                "type": "Bear Call"
            },
            {
                "url": "https://optionrecom.com/bull-put-spread-strategy/",
                "type": "Bull Put"
            }
        ]
        
        # Process each strategy page
        total_records = 0
        total_skipped = 0
        
        for strategy in strategies:
            records, skipped = process_strategy_page(driver, strategy["url"], strategy["type"], conn, cursor)
            total_records += records
            total_skipped += skipped
        
        print(f"\nTotal records saved to database: {total_records}")
        if total_skipped > 0:
            print(f"Total records skipped due to None ticker values: {total_skipped}")
        
        # If we want to verify no None values made it to the database
        cursor.execute("SELECT COUNT(*) FROM option_strategies WHERE ticker IS NULL OR ticker = 'None' OR ticker = 'N/A' OR ticker = ''")
        none_count = cursor.fetchone()[0]
        if none_count > 0:
            print(f"WARNING: There are still {none_count} records with empty or None ticker values in the database!")
        else:
            print("Verification successful: No records with None ticker values in the database.")
        
        # Query to show what was saved
        cursor.execute("SELECT strategy_type, tab_name, COUNT(*) as count FROM option_strategies GROUP BY strategy_type, tab_name")
        results = cursor.fetchall()
        
        print("\nRecords by strategy and tab:")
        for strategy, tab, count in results:
            print(f"  {strategy} - {tab}: {count} records")
        
        return total_records
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return 0
    finally:
        driver.quit()
        conn.close()

if __name__ == "__main__":
    scrape_option_strategies()

Starting login process...

Please log in manually. Waiting 30 seconds...

===== Processing Bear Call Strategy Page =====
Processing Tab 1: 'Mild Risk 95-97% accuracy > shorter expiry'
Saved 10 records from tab #1
Processing Tab 2: 'Minimal Risk 97-99% accuracy > shorter expiry'
Saved 10 records from tab #2
Processing Tab 3: 'Mild Risk 95-97% accuracy > longer expiry'
Saved 10 records from tab #3
Processing Tab 4: 'Minimal Risk 97-99% accuracy > longer expiry'
Saved 10 records from tab #4

===== Processing Bull Put Strategy Page =====
Processing Tab 1: 'Mild Risk 95-97% accuracy > shorter expiry'
Saved 10 records from tab #1
Processing Tab 2: 'Minimal Risk 97-99% accuracy > shorter expiry'
Saved 10 records from tab #2
Processing Tab 3: 'Mild Risk 95-97% accuracy > longer expiry'
Saved 10 records from tab #3
Processing Tab 4: 'Minimal Risk 97-99% accuracy > longer expiry'
Saved 10 records from tab #4

Total records saved to database: 80

Records by strategy and tab:
  Bear Call - Mild Ri